In [1]:
import plotly.graph_objs as go
import random
import numpy as np
import torch
from differentiable_DR_model import DRagent
from baseline import MLP, RNNmodel
import keras

/Users/bb/opt/anaconda3/envs/dr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for i in range(2):
    df_dp = np.load(f"dataset/co50/data_N30_{i}.npz")
    df_dp2 = np.load(f"dataset/co50/data_N310_{i}.npz")
    newp = np.concatenate([df_dp['p'], df_dp2['p']])
    newd = np.concatenate([df_dp['d'], df_dp2['d']])
    newprice = np.concatenate([df_dp['price'], df_dp2['price']])
    paras = df_dp['paras']
    np.savez(f"dataset/co50/data_N330_{i}", paras = paras, price = newprice, d=newd, p=newp)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/co50/data_N30_0.npz'

In [ ]:
dataset = np.load(f"../EnergyStorage/dataset/data_N365_1.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 24
model = DRagent(P1, P2, T)
model.load_state_dict(torch.load('../EnergyStorage/result/model_gradient/model_1.pth'))
model.eval()

# plot validation loss

In [39]:
val_loss_mlp = np.zeros((10, 500))
val_loss_rnn = np.zeros((10, 500))
val_loss_model = np.zeros((10, 500))
val_loss_quad = np.zeros((10, 500))

for i in range(10):
    val_loss_mlp[i] = np.load(f'result/baseline/version2/MLP_loss_{i}_train200test10.npz')['val_loss'][:500]
    val_loss_rnn[i] = np.load(f'result/baseline/version2/RNN_loss_{i}_train200test10.npz')['val_loss'][:500]
for i in range(10):
    val_loss_model[i] = np.load(f'result/model_gradient/ICNN_vector/version2/loss_{i}.npz')['val_loss']
    val_loss_quad[i] = np.load(f'result/model_gradient/quad/version2/loss_{i}.npz')['val_loss']

In [31]:
val_loss_rnn[1][-1], val_loss_model[1][-1]

(0.08439145982265472, 0.006361876411798915)

In [ ]:
 val_loss_model[0][-10:], val_loss_rnn[0][-10:]

In [45]:
fig = go.Figure()
x1 = np.linspace(0,500,100)
x2 = np.linspace(0,500,500)

median_NN =  np.median(val_loss_mlp, axis=0)[:500]
median_NN_2 = np.quantile(val_loss_mlp,0.2, axis=0)[:500]
median_NN_8 = np.quantile(val_loss_mlp,0.8,axis=0)[:500]

median_RNN =  np.median(val_loss_rnn, axis=0)[:500]
median_RNN_2 = np.quantile(val_loss_rnn,0.2,axis=0)[:500]
median_RNN_8 = np.quantile(val_loss_rnn,0.8,axis=0)[:500]

median_opt1 =  np.median(val_loss_model, axis=0)[:500]
median_opt1_2 = np.quantile(val_loss_model, 0.2, axis=0)[:500]
median_opt1_8 = np.quantile(val_loss_model, 0.8, axis=0)[:500]

median_quad1 =  np.median(val_loss_quad, axis=0)[:500]
median_quad1_2 = np.quantile(val_loss_quad, 0.2, axis=0)[:500]
median_quad1_8 = np.quantile(val_loss_quad, 0.8, axis=0)[:500]


fig.add_trace(go.Scatter(x=x1, y=median_NN_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False ))
fig.add_trace(go.Scatter(x=x1,y=median_NN_8,fill='tonexty', mode='lines',  fillcolor='rgba(203,158,252,0.2)',line_color = 'rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x1, y=median_RNN_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False ))
fig.add_trace(go.Scatter(x=x1,y=median_RNN_8,fill='tonexty', mode='lines',  fillcolor='rgba(168,236,217,0.5)',line_color='rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x2, y=median_opt1_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False    ))
fig.add_trace(go.Scatter(x=x2,y=median_opt1_8,fill='tonexty', mode='lines',  fillcolor='rgba(240,96,72,0.5)',line_color = 'rgba(0,0,0,0)',showlegend = False))


fig.add_trace(go.Scatter(x=x2, y=median_quad1_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False    ))
fig.add_trace(go.Scatter(x=x2,y=median_quad1_8,fill='tonexty', mode='lines',  fillcolor='rgba(248,248,51,0.2)',line_color = 'rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x1,y=median_NN, mode='lines',  line_color='#b681f0',opacity=0.5, line_width=3,name = 'median(MLP,200)')) 
fig.add_trace(go.Scatter(x=x1,y=median_RNN, mode='lines',  line_color='#07cd98',opacity=0.7,line_width=3,name = 'median(RNN,200)')) 
fig.add_trace(go.Scatter(x=x2,y=median_opt1, mode='lines',  line_color='#f06048',opacity=0.5,line_width=3,name = 'median(Nonconvex,40)')) 
fig.add_trace(go.Scatter(x=x2,y=median_quad1, mode='lines',  line_color='#ffe249',opacity=0.5,line_width=3,name = 'median(Quad,40)')) 

fig.update_layout({#'title': 'NRMSE comparison',
    'xaxis':{'title':'Iterations', 'showline':True},
    'yaxis':{'title': 'Validation Loss',"range":[0,0.1]},
    "font": {"size": 40, 'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
   'height':900
})

fig.show()


In [ ]:
fig.write_image("result/image/validation_loss.pdf")

In [ ]:
dataset = np.load(f"dataset/co10/data_N210_0.npz")
dataset['paras']

In [ ]:
for i in range(10):
    dataset = np.load(f"dataset/data_N365_{i}.npz")
    print(i, dataset['paras'])
for i in range(10):
    dataset = np.load(f"../EnergyStorage/dataset/data_N365_{i}.npz")
    print(i, dataset['paras'])


# Plot Prediction

In [ ]:
dataset = np.load(f"dataset/version2/data_N840_0.npz")
p = dataset['p']
d = dataset['d']
dr = p-d
fig = go.Figure()
i = np.random.randint(10)
fig.add_trace(go.Scatter(y=dr[i]))
#fig.add_trace(go.Scatter(y=e[i]))
fig.show()

In [ ]:
np.load(f'result/model_gradient/ICNN_vector/version3/loss42.npz')['val_loss'][-5:]

In [ ]:
np.load(f"result/model_gradient/ICNN_vector/version3/loss11`.npz")['val_loss'][-5:]

In [32]:

dataset = np.load(f"dataset/version2/data_N240_8.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 6
#rnnmodel = keras.models.load_model('result/baseline/version2/model_RNN_0')
#mlpmodel = keras.models.load_model('result/baseline/version2/model_MLP_0')

In [33]:
dataset['paras']

array([[0.5 , 0.5 , 0.  , 6.05, 2.  , 0.94]])

In [35]:
from quadratic_model import DRagent_Quad
model2 = DRagent_Quad(P1, P2, T, 0.91)
model2.load_state_dict(torch.load('result/model_gradient/quad/version2/model_0.pth'))
model2.eval()

DRagent_Quad(
  (layer): OptLayer()
)

In [6]:
model = DRagent(P1, P2, T, eta=0.94,type='vector')
model.load_state_dict(torch.load('result/model_gradient/ICNN_vector/version2/model_0.pth'))
model.eval()

DRagent(
  (costNN): ICNN(
    (linear_z1): Linear(in_features=24, out_features=1, bias=False)
    (linear_y0): Linear(in_features=6, out_features=24, bias=True)
    (linear_y1): Linear(in_features=6, out_features=1, bias=True)
    (act): Softplus(beta=1, threshold=20)
  )
  (layer): OptLayer()
)

In [7]:
test_price = dataset['price'][:40]
df_dp = dataset
d2 = df_dp["d"][:40]
p2 = df_dp["p"][:40]
test_dr = p2-d2
price_tensor2 = torch.from_numpy(test_price).double()
d_tensor2 = torch.from_numpy(d2).double()
p_tensor2 = torch.from_numpy(p2).double()
pred_d2, pred_p2 = model(price_tensor2, d_tensor2, 0)
pred_d2 = pred_d2.detach().numpy()
pred_p2 = pred_p2.detach().numpy()
pred_dr = pred_p2-pred_d2

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-27


/Users/bb/opt/anaconda3/envs/dr/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:178: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


In [10]:
pred_dr = rearrange(pred_dr)
test_dr = rearrange(p2-d2)
test_price = rearrange(test_price)

0
1
2
3
10
11
12
13
20
21
22
23
30
31
32
33
40
41
42
43
50
51
52
53
60
61
62
63
70
71
72
73
80
81
82
83
90
91
92
93
0
1
2
3
10
11
12
13
20
21
22
23
30
31
32
33
40
41
42
43
50
51
52
53
60
61
62
63
70
71
72
73
80
81
82
83
90
91
92
93
0
1
2
3
10
11
12
13
20
21
22
23
30
31
32
33
40
41
42
43
50
51
52
53
60
61
62
63
70
71
72
73
80
81
82
83
90
91
92
93


In [11]:
pred_d2, pred_p2 = model2(price_tensor2)
pred_d2 = pred_d2.detach().numpy()
pred_p2 = pred_p2.detach().numpy()
pred_dr2 = pred_p2-pred_d2
pred_dr2 = rearrange(pred_dr2)

0
1
2
3
10
11
12
13
20
21
22
23
30
31
32
33
40
41
42
43
50
51
52
53
60
61
62
63
70
71
72
73
80
81
82
83
90
91
92
93


In [9]:
def rearrange(dr,T=6):
    cnt = dr.shape[0]//(24//T)
    res = np.zeros((cnt,24))
    for i in range(cnt):
        for j in range(24//T):
            print(i*cnt+j)
            res[i][j*T:(j+1)*T] = dr[i*24//T+j][:]
    return res

In [18]:
def predict(model,T=6):
    res = np.zeros((5,24))
    for i in range(5):
        for j in range(24//T):
            res[i][j*T:(j+1)*T] = model(dataset['price'][i*24//T+j:i*24//T+j+1])
    return res

In [37]:
for i in range(2):
    dr1 = predict(rnnmodel)[i]
    dr3 = predict(mlpmodel)[i]

    t1 = go.Scatter(x=np.arange(24), y=test_price[i],mode='lines', name='price',line = {'width':6,'dash':'dash','color':'#ffa05a'},yaxis='y2')
    t2 = go.Scatter(x=np.arange(24), y=test_dr[i],mode='lines', name='True',line = {'width':6,'dash':'dash'})
    t3 = go.Scatter(x=np.arange(24), y=dr1,mode='lines', name='RNN',line = {'width':4})
    t4 = go.Scatter(x=np.arange(24), y=dr3,mode='lines', name='MLP',line = {'width':4})
    t5 = go.Scatter(x=np.arange(24), y=pred_dr[i],mode='lines', name='Ours(non-convex)',line = {'width':4, 'color':'red'})
    t6 = go.Scatter(x=np.arange(24), y=pred_dr2[i],mode='lines', name='Ours(quadratic)',line = {'width':4, 'color':'#8dbf79'})
    fig = go.Figure(data=[t2, t5, t6, t3, t4, t1],layout={
        'xaxis':{'title':'Hour', 'showline':True},
        'yaxis':{'title': 'Dispatch(MW)','range':[-0.8,0.5]},
        'yaxis2': {'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[1,100], 'showgrid':False},
        "font": {"size": 40,'color':'#3a4142'},
        'template':'plotly_white',
        'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
        'width':1600,
        'height':900
    })
    fig.show()

In [ ]:
fig.write_image("result/image/predict_compare_quadratic.pdf")

In [ ]:
dataset = np.load(f"dataset/data_N365_1.npz")
dataset['paras']

In [ ]:
dataset['paras'],dataset2['paras']

# Compare with Non-convex and convex

In [ ]:
dataset = np.load(f"dataset/version3/data_N210_42.npz")
dr = dataset['p']-dataset['d']
dataset2 = np.load(f"dataset/nod/data_N50_0_convex.npz")
dr2 = dataset2['p'] - dataset2['d']
i = np.random.randint(0,30)
print(i)
fig = go.Figure()
fig.add_trace(go.Scatter(y=dr[i],name='non-convex1'))
fig.add_trace(go.Scatter(y=dr2[i],name='convex'))
#fig.add_trace(go.Scatter(y=dataset['price'][i],name='price',yaxis="y2"))

In [ ]:
def compute(dataset, model, model_v, i):
    price = dataset['price'][i]
    p, d = dataset['p'], dataset['d']
    d_tensor = torch.from_numpy(d[i:i+3]).double()
    d_tensor_scalar = d_tensor.permute(1,0).unsqueeze(2)
    dr = p[i]-d[i]
    e = np.zeros(25)
    e[0] = dataset['paras'][0][2]*0.5
    eta = dataset['paras'][0][3]
    for t in range(1,25):
        e[t] = e[t-1] + p[i][t-1] * eta - d[i][t-1] / eta
    d = d[i]
    Truecoeff = [(1-e[i])*(1-e[i])*d[i] for i in range(24)]
    c2, c1, _ = model.approximate_cost2(d_tensor_scalar, diff=False)
    c1, c2 = c1.detach().numpy()[0], c2.detach().numpy()[0]
    Predict = [c1[i]*d[i]+c2[i]*d[i]*d[i] for i in range(24)]

    # c2, c1 = model_v.approximate_cost(d_tensor, diff=False)
    # c1, c2 = c1.detach().numpy()[0], np.diag(c2.detach().numpy()[0])
    # Predict2 = [c1[i]*d[i]+c2[i]*d[i]*d[i] for i in range(24)]

    print('True',Truecoeff)
    print('Predict',Predict)

In [ ]:
dataset = np.load(f"dataset/co/data_N50_0_co100.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 24
model = DRagent(P1, P2, T, type='scalar')
model.load_state_dict(torch.load('result/model_gradient/ICNN/model_2.pth'))
model2 = DRagent(P1, P2, T, type='vector')
model2.load_state_dict(torch.load('result/model_gradient/ICNN_vector/model_0.pth'))
i = np.random.randint(0,40)
compute(dataset, model, model, i)

In [ ]:
val_loss_model = np.zeros((10,500))
for i in range(2):
    val_loss_model[i] = np.load(f'result/model_gradient/loss_{i}.npz')['val_loss']
val_loss_model[:,-1]

In [38]:
for name in ['MLP','RNN']:
    for i in range(1,10):
        print(np.load(f'result/baseline/version2/{name}_loss_{i}_train200test10.npz')['val_loss'][-5:])

[0.09201612 0.09198807 0.09196243 0.09193788 0.09190972]
[0.02514977 0.02513967 0.02512919 0.02511872 0.02511041]
[0.04296265 0.04295345 0.04294468 0.04293554 0.04292469]
[0.01991783 0.01991883 0.01991958 0.0199202  0.01992036]
[0.02655785 0.02655733 0.02655467 0.02655233 0.02654869]
[0.04546321 0.04546397 0.04546447 0.04546652 0.0454605 ]
[0.02465345 0.02465055 0.02464984 0.0246483  0.02464381]
[0.02575568 0.02575756 0.02575243 0.02574975 0.02574218]
[0.03538007 0.03538239 0.03538574 0.03538574 0.03537854]
[0.08360572 0.0856403  0.08542471 0.08447798 0.08439146]
[0.02046777 0.02051509 0.02066188 0.02063522 0.02058065]
[0.03503226 0.03482347 0.03485461 0.03508733 0.03504541]
[0.01659124 0.01641317 0.01637678 0.01625728 0.01640598]
[0.0169177  0.01680934 0.01650681 0.01681679 0.01743026]
[0.03716395 0.03694345 0.03694494 0.03711333 0.03734111]
[0.02839189 0.02850396 0.02871846 0.0287801  0.02870596]
[0.02256689 0.02254765 0.0224711  0.02230117 0.02213909]
[0.0249367  0.0249234  0.024973

In [41]:
for i in range(1,10):
    print(np.load(f'result/model_gradient/ICNN_vector/version2/loss_{i}.npz')['val_loss'][-1:])
    print(np.load(f'result/model_gradient/quad/version2/loss_{i}.npz')['val_loss'][-1:])

[0.00636188]
[0.011073]
[0.00857307]
[0.02915914]
[0.01675702]
[0.02942078]
[0.02300727]
[0.04633959]
[0.0372839]
[0.04850031]
[0.01361291]
[0.03252608]
[0.01968697]
[0.03141404]
[0.01040708]
[0.04046429]
[0.0288936]
[0.03483039]
